## crop and resize Div2k dataset:
1. 找出所有高度或宽度大于等于2040的图片， 如果高度>宽度的， 旋转90度
2. 在基础上， 找出所有宽度大于等于1152的图片
3. CROP， 宽度2040, 高度1152
4. 保存 变换好的图片（维持原来的PNG格式）， 图片质量不要压缩， 保存在HR子目录中
5. 在以上的基础上把图片高度和宽度各为原来的1/4, RESIZE， 图片压缩用CV2的默认压缩率， 图片保存在LR子目录下

In [3]:
import os
from PIL import Image
import cv2
import numpy as np

def pil_to_cv(image):
    # Convert PIL image to OpenCV format (BGR)
    return cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

def cv_to_pil(image):
    # Convert OpenCV image to PIL format (RGB)
    return Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

def process_images(directory_path, hr_output_directory, lr_output_directory):
    if not os.path.exists(hr_output_directory):
        os.makedirs(hr_output_directory)
    if not os.path.exists(lr_output_directory):
        os.makedirs(lr_output_directory)

    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith('.png'):
                try:
                    with Image.open(file_path) as img:
                        width, height = img.size

                        if (height >= 2040 and width>=1152) or (width >= 2040 and height >=1152):
                            if height > width:
                                img = img.rotate(90, expand=True)
                                width, height = img.size

                        if width >= 2040 and height >= 1152:
                            # 裁剪的目标尺寸
                            new_width = 2040
                            new_height = 1152

                            # 计算裁剪区域
                            left = (width - new_width) / 2
                            top = (height - new_height) / 2
                            right = (width + new_width) / 2
                            bottom = (height + new_height) / 2

                            # 裁剪图像
                            img_cropped = img.crop((left, top, right, bottom))

                            # 保存裁剪后的高分辨率图像
                            hr_output_path = os.path.join(hr_output_directory, file)
                            img_cropped.save(hr_output_path, format='PNG')

                            # 将 PIL 图像转换为 OpenCV 格式以便缩放
                            img_cv = pil_to_cv(img_cropped)

                            # 缩小图像（缩小 4 倍）
                            new_size = (new_width // 4, new_height // 4)
                            img_resized = cv2.resize(img_cv, new_size, interpolation=cv2.INTER_LINEAR)

                            # 将缩小后的图像转换回 PIL 格式以便保存
                            img_resized_pil = cv_to_pil(img_resized)

                            # 保存缩小后的低分辨率图像
                            lr_output_path = os.path.join(lr_output_directory, file)
                            img_resized_pil.save(lr_output_path, format='PNG')


                except Exception as e:
                    print(f"Error processing {file_path}: {e}")


# Define the paths
directory_path = '/home/royliu/Documents/datasets/Flickr2K/Flickr2K_HR'
hr_output_directory = os.path.join(directory_path, '../own/train/HR')
lr_output_directory = os.path.join(directory_path, '../own/train/LR')

# Process images
process_images(directory_path, hr_output_directory, lr_output_directory)
print('Done!')

Done!


## check images sizes in the folder

In [1]:
import os
from PIL import Image

def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size  # Returns a tuple (width, height)

def list_unique_image_sizes(directory_path):
    sizes = set()
    
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                try:
                    size = get_image_size(file_path)
                    sizes.add(size)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
    
    return sizes

directory_path = '/home/royliu/Documents/datasets/Flickr2K/val/HR'
unique_sizes = list_unique_image_sizes(directory_path)

print("唯一的图片尺寸:")
for size in sorted(unique_sizes):
    print(f"宽度: {size[0]} 高度: {size[1]}")


唯一的图片尺寸:
宽度: 1140 高度: 2040
宽度: 1428 高度: 2028
宽度: 1524 高度: 2040
宽度: 1596 高度: 1200
宽度: 1608 高度: 1200
宽度: 1956 高度: 2040
宽度: 2040 高度: 1344
宽度: 2040 高度: 1356
宽度: 2040 高度: 1524
宽度: 2040 高度: 1584
宽度: 2040 高度: 2040


In [4]:
import os
from PIL import Image

def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size  # Returns a tuple (width, height)

def list_unique_image_sizes(directory_path):
    sizes = set()
    
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                try:
                    size = get_image_size(file_path)
                    sizes.add(size)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
    
    return sizes

directory_path = '/home/royliu/Documents/datasets/Flickr2K/own/train/HR'
unique_sizes = list_unique_image_sizes(directory_path)

print("唯一的图片尺寸:")
for size in sorted(unique_sizes):
    print(f"宽度: {size[0]} 高度: {size[1]}")


唯一的图片尺寸:
宽度: 2040 高度: 1152
